In [ ]:
from pathlib import Path

from utils.base_directory import base_directory
from utils.dataset_utils import get_leaning_datasets, leaning_no_center_label_mapping, \
    leaning_with_center_label_mapping
from utils.model_utils import evaluate_models, get_custom_leaning_models

In [ ]:
GET_DATASETS = lambda: get_leaning_datasets()
DATASET_SAMPLE_SIZE = 1_000

datasets_with_center = []
for dataset in GET_DATASETS():
    dataset = dataset.take_even_class_distribution_sample(DATASET_SAMPLE_SIZE)
    datasets_with_center.append(dataset.transform_for_inference(leaning_with_center_label_mapping).to_huggingface())

datasets_without_center = []
for dataset in GET_DATASETS():
    dataset.dataframe = dataset.dataframe[dataset.dataframe["leaning"] != "center"]
    dataset.dataframe["leaning"] = dataset.dataframe["leaning"].cat.remove_unused_categories()

    dataset = dataset.take_even_class_distribution_sample(DATASET_SAMPLE_SIZE)
    datasets_without_center.append(dataset.transform_for_inference().to_huggingface())

In [ ]:
# Lambda, so that the generator can be reused.
GET_MODELS = lambda: get_custom_leaning_models(Path("dataset_benchmark") / "leave_one_in" / "political_leaning")


def get_filtered_models(supports_center_leaning_class: bool):
    yield from filter(
        lambda model: model.supports_center_leaning_class == supports_center_leaning_class,
        GET_MODELS(),
    )


results_with_center = evaluate_models(
    lambda: get_filtered_models(True),
    datasets_with_center,
)

In [ ]:
results_with_center.count

In [ ]:
results_with_center.accuracy

In [ ]:
results_with_center.f1

In [ ]:
results_with_center.precision

In [ ]:
results_with_center.recall

In [ ]:
results_with_center.save_confusion_matrix_images(
    Path(
        base_directory,
        "analysis",
        "model_evaluation",
        "political_leaning",
        "confusion_matrix_results",
        "with_center",
    ),
    leaning_with_center_label_mapping
)

In [ ]:
results_without_center = evaluate_models(
    lambda: get_filtered_models(False),
    datasets_without_center,
)

In [ ]:
results_without_center.count

In [ ]:
results_without_center.accuracy

In [ ]:
results_without_center.f1

In [ ]:
results_without_center.precision

In [ ]:
results_without_center.recall

In [ ]:
results_without_center.save_confusion_matrix_images(
    Path(
        base_directory,
        "analysis",
        "model_evaluation",
        "political_leaning",
        "confusion_matrix_results",
        "no_center",
    ),
    leaning_no_center_label_mapping
)